# PUC Rio | MVP | Machine Learning

Aluno: Joel Carneiro Dutra





## **Definição do Problema:**

Uma empresa que possui várias franquias de lojas, precisa prever a quantidade de produtos que cada franquia precisará para manter seus estoques otimizados. O objetivo é garantir que cada franquia tenha o estoque adequado para atender à demanda de seus clientes, minimizando ao mesmo tempo os custos de armazenamento e o risco de falta de produtos.

### **Passo 1: Coleta de Dados**

A empresa coleta dados históricos de vendas e estoque de cada uma das franquias, juntamente com informações sobre sazonalidade, promoções, dados geográficos, entre outros fatores que podem influenciar a demanda.

Esta etapa foi realizada por meio de uma extração no Google BigQuery, contendo os seguintes campos:

- **dt_venda** - Data em que a venda foi realizada
- **loja** - Código das franquias que realizaram as vendas
- **uf** - UF das franquias (contendo Rio de Janeiro e São Paulo)
- **produto** - Descrição do produto vendido (Produto X, Y e Z)
- **canal_venda** - Canal onde ocorreu a venda (Loja ou Site)
- **tipo_venda** - Tipo de venda (Promoção ou Regular)
- **vlr_venda** - Valor total da venda
- **qt_venda** - Quantidade de itens vendidos

Atributos:
- **qt_dias_com_estoque** - Quantidade de dias em que a loja tinha estoque
- **qt_dias_sem_estoque** - Quantidade de dias em que a loja não tinha estoque
- **qt_dias_com_estoque_aberta** - Quantidade de dias em que a loja tinha estoque e estava aberta para venda
- **qt_dias_sem_estoque_aberta** - Quantidade de dias em que a loja não tinha estoque e estava aberta para venda
- **qt_dias_loja_fechada** - Quantidade de dias em que a loja esteve fechada
- **estoque_loja** - Quantidade de estoque da loja
- **habilitador** - Determina se a loja deverá ser abastecida caso tenha menos que 5 itens em seu estoque (variável 1 ou 0)

In [ ]:
# Configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

# Imports necessários
import requests
from io import StringIO
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as ms
from matplotlib import cm
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

### **Passo 2: Carga dos Dados**

Nessa etapa fazemos a conexão com o dataset, fazendo a requisição do arquivo em formato .csv e a leitura do arquivo em um DataFrame.

`Opção de leitura #1 - Origem GitHub`

In [ ]:
url = 'https://raw.githubusercontent.com/joel-c-dutra/mvp-ml/main/base_historica.csv'

# Leitura do arquivo CSV em um DataFrame
page = requests.get(url)

dataset = pd.read_csv(StringIO(page.text))
dataset.head()

In [ ]:
# Informa a URL de importação do dataset
url = "https://raw.githubusercontent.com/joel-c-dutra/mvp-ml/main/base_historica.csv"

# Informa o cabeçalho das colunas
colunas = ['dt_venda', 'loja', 'uf', 'produto', 'canal_venda', 'tipo_venda', 'vlr_venda', 'qt_venda', 'qt_dias_com_estoque', 'qt_dias_sem_estoque', 'qt_dias_com_estoque_aberta', 'qt_dias_sem_estoque_aberta', 'qt_dias_loja_fechada', 'estoque_loja', 'habilitador']

# Lê o arquivo utilizando as colunas informadas
dataset = pd.read_csv(url, names=colunas, skiprows=1, delimiter=',')
dataset.head()

`Opção de leitura #2 - Origem Google Drive`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

caminho_arquivo = '/content/drive/My Drive/datasets/base_historica.csv'

# Leitura do arquivo CSV em um DataFrame
dataset = pd.read_csv(caminho_arquivo)
dataset.head()

In [ ]:
# Referenciando os atribuitos a uma nova variável
dataset_atributos = dataset.loc[:, ['qt_dias_com_estoque', 'qt_dias_sem_estoque', 'qt_dias_com_estoque_aberta', 'qt_dias_sem_estoque_aberta', 'qt_dias_loja_fechada', 'habilitador']]
dataset_atributos.head()

### **Passo 3: Análise dos Dados**

Nesta etapa é apresentado algumas informações do dataset, tais como:

- Volume de registros
- Tipo de dado de cada atributo
- Descrição dos campos

**3.1 - Volume de registros**

In [ ]:
# Mostra as dimensões do dataset
print(dataset.shape)

**3.2 - Tipo de dado de cada atributo**

In [ ]:
# Mostra as informações do dataset
print(dataset.info())

In [ ]:
# Verifica o tipo de dataset de cada atributo
dataset.dtypes

**3.3 - Descrição dos campos**

In [ ]:
# Faz um resumo estatístico do dataset (média, desvio padrão, mínimo, máximo e os quartis)
dataset.describe()

### **Passo 4: Pré-Processamento dos Dados**



Nesta etapa realizamos os seguintes processos:

- Verificação por campos nulos;
- Distribuição da classe;
- Histograma e Matriz de Correlação;
- Feature Selection.

**4.1 - Campos nulos**

In [ ]:
# verificando nulls no dataset
dataset.isnull().sum()

**4.2 - Distribuição da classe**

In [ ]:
# distribuição da classe
print(dataset.groupby('habilitador').size())

In [ ]:
# Referenciando os atribuitos a uma nova variável
dataset_atributos = dataset.loc[:, ['qt_dias_com_estoque', 'qt_dias_sem_estoque', 'qt_dias_com_estoque_aberta', 'qt_dias_sem_estoque_aberta', 'qt_dias_loja_fechada', 'habilitador']]
dataset_atributos.head()

**4.3 - Histograma e Matriz de Correlação**

In [ ]:
# Histograma
dataset_atributos.hist(figsize = (15,10))
plt.show()

In [ ]:
# Density Plot
dataset_atributos.plot(kind = 'density', subplots = True, layout = (3,3), sharex = False, figsize = (15,10))
plt.show()

In [ ]:
# Matriz de Correlação com Matplotlib Seaborn
sns.heatmap(dataset_atributos.corr(), annot=True, cmap='RdBu');

Na etapa acima (Matriz de Correlação) já é possível analisar os atributos mais prováveis para utilização no(s) modelo(s).

Os mesmos são apresentados também no processo a seguir (Feature Selection).

**4.4 - Feature Selection**

In [ ]:
# Preparação dos dados para Feature Selection

# Separação em bases de treino e teste (holdout)
array = dataset_atributos.values
X = array[:,0:5] # atributos
y = array[:,5]   # classe (target)

In [ ]:
# SelectKBest

# Seleção de atributos com SelectKBest
best_var = SelectKBest(score_func=f_classif, k=3)

# Executa a função de pontuação em (X, y) e obtém os atributos selecionados
fit = best_var.fit(X, y)

# Reduz X para os atributos selecionados
features = fit.transform(X)

# Resultados
print('\nNúmero original de atributos:', X.shape[1])
print('\nNúmero reduzido de atributos:', features.shape[1])

# Exibe os atributos orginais
print("\nAtributos Originais:", dataset_atributos.columns[0:6])

# Exibe as pontuações de cada atributos e os 4 escolhidos (com as pontuações mais altas)
np.set_printoptions(precision=3) # 3 casas decimais
print("\nScores dos Atributos Originais:", fit.scores_)
print("\nAtributos Selecionados:", best_var.get_feature_names_out(input_features=dataset_atributos.columns[0:5]))

In [ ]:
# Eliminação Recursiva de Atributos

# Criação do modelo
modelo = LogisticRegression(max_iter=200)

# Eliminação Recursiva de Atributos
rfe = RFE(modelo, n_features_to_select=3)
fit = rfe.fit(X, y)

# Print dos resultados
print("Atributos Originais:", dataset_atributos.columns[0:6])

# Exibe os atributos selecionados (marcados como True em "Atributos Selecionados"
# e com valor 1 em "Ranking dos Atributos")
print("\nAtributos Selecionados: %s" % fit.support_)
print("\nRanking de atributos: %s" % fit.ranking_)
print("\nQtd de melhores Atributos: %d" % fit.n_features_)
print("\nNomes dos Atributos Selecionados: %s" % fit.get_feature_names_out(input_features=dataset_atributos.columns[0:5]))

Ao utilizar o **SelectKBest** e a **Eliminação Recursiva**, foi possível determinar dois campos em comum em ambas as análises:
- *qt_dias_sem_estoque*
- *qt_dias_com_estoque_aberta*

Sendo assim, serão utilizados esses dois atributos no processo de treino e teste do(s) modelo(s).

### **Passo 5: Separação em conjunto de treino e teste**

Nesta etapa foram realizados os seguintes processos:

- Holdout (com estratificação)
- Teste e comparação dos modelos
- Validação cruzada (Pipeline)

**5.1 - Holdout com estratificação**

In [ ]:
test_size = 0.20 # tamanho do conjunto de teste
seed = 7 # semente aleatória

# Separação em conjuntos de treino e teste
array = dataset_atributos.values
X = array[:,[1,2]]
y = array[:,5]

X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=test_size, shuffle=True, random_state=seed, stratify=y) # holdout com estratificação

# Parâmetros e partições da validação cruzada
scoring = 'accuracy'
num_particoes = 10
kfold = StratifiedKFold(n_splits=num_particoes, shuffle=True, random_state=seed) # Validação cruzada

**5.2 - Teste e comparação dos modelos**

In [ ]:
np.random.seed(7) # definindo uma semente global

# Lista que armazenará os módulos
models = []

# Criando os modelos e adicionando-os na lista de modelos
models.append(('CART', DecisionTreeClassifier()))
models.append(('SVM', SVC()))

# Definindo os parâmetros do classificador base o BaggingClassifier
base = DecisionTreeClassifier()
num_trees = 4
max_features = 2

# Criando os modelos para o VotingClassifier
bases = []
model1 = DecisionTreeClassifier()
bases.append(('cart', model1))
model2 = SVC()
bases.append(('svm', model2))

In [ ]:
# Criando os ensembles e adicionando-os na lista de modelos
models.append(('RF', RandomForestClassifier(n_estimators=num_trees, max_features=max_features)))
models.append(('ET', ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features)))
models.append(('GB', GradientBoostingClassifier(n_estimators=num_trees)))

In [ ]:
# Lista para armazenar os resultados
results = []
names = []

# Avaliação dos modelos
for name, model in models:
  cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

Com base nos resultados apresentados, foi possível analisar que o modelo **Random Forest** foi o que apresentou o melhor resultados na comparação entre os modelos, com os seguintes resultados:


- RF:      0.723891 (0.029982)
- CART:    0.723200 (0.021741)
- ET:      0.723200 (0.021741)
- GB:      0.656366 (0.017241)
- SVM:     0.630786 (0.017767)


In [ ]:
# Boxplot de comparação dos modelos
fig = plt.figure(figsize=(20,5))
fig.suptitle('Comparação dos Modelos')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

**5.3 - Validação cruzada (Pipeline)**

Após a escolha do modelo, foi feita a comparação entre os dados:

- Original
- Padronizado
- Normalizado

In [ ]:
np.random.seed(7) # definindo uma semente global para este bloco

# Lista para armazenar os pipelines e os resultados para todas as visões do dataset
pipelines = []
results = []
names = []

# Criando os elementos do pipeline

random_forest = ('RF', RandomForestClassifier(n_estimators=num_trees, max_features=max_features))

# Transformações que serão utilizadas
standard_scaler = ('StandardScaler', StandardScaler())
min_max_scaler = ('MinMaxScaler', MinMaxScaler())

In [ ]:
#Montando os pipelines

# Dataset original
pipelines.append(('RF-orig', Pipeline([random_forest])))

# Dataset padronizado
pipelines.append(('RF-padr', Pipeline([standard_scaler, random_forest])))

# Dataset normalizado
pipelines.append(('RF-norm', Pipeline([min_max_scaler, random_forest])))

# Executando os pipelines
for name, model in pipelines:
  cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %.3f (%.3f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

Com base nos resultados obtidos, nota-se uma pequena variação entre os dados.

- RF-orig: 0.729 (0.027)
- RF-padr: 0.723 (0.022)
- RF-norm: 0.726 (0.023)

In [ ]:
# Boxplot de comparação dos modelos
fig = plt.figure(figsize=(20,6))
fig.suptitle('Comparação dos Modelos - Dataset original, padronizado e normalizado')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names, rotation=90)
plt.show()

### **Passo 6: Avaliação do modelo**

Nesta etapa foi realizada a avaliação do modelo em conjunto de teste

In [ ]:
# Avaliação do modelo com o conjunto de testes

# Preparação do modelo
scaler = StandardScaler().fit(X_train) # ajuste do scaler com o conjunto de treino
rescaledX = scaler.transform(X_train) # aplicação da padronização no conjunto de treino
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
model.fit(rescaledX, y_train)

# Estimativa da acurácia no conjunto de teste
rescaledTestX = scaler.transform(X_test) # aplicação da padronização no conjunto de teste
predictions = model.predict(rescaledTestX)
print(accuracy_score(y_test, predictions))

Com base no resultado obtido, verificou-se que a acurácia foi de **72,99%**.

### **Passo 7: Aplicação em produção**

Nesta etapa, após a escolha do modelo, foi realizado o uso do mesmo com novos dados

In [ ]:
# Preparação do modelo com todo o dataset
scaler = StandardScaler().fit(X) # ajuste do scaler com todo o dataset
rescaledX = scaler.transform(X) # aplicação da padronização com todo o dataset
model.fit(rescaledX, y)

In [ ]:
# Simulação com novos dados

data = {
        'qt_dias_sem_estoque': [3, 5, 0],
        'qt_dias_com_estoque_aberta': [7, 28, 10],
        }

atributos = ['qt_dias_sem_estoque', 'qt_dias_com_estoque_aberta']
entrada = pd.DataFrame(data, columns=atributos)

array_entrada = entrada.values
X_entrada = array_entrada[:,0:2].astype(float)

# Padronização nos dados de entrada usando o scaler utilizado em X
rescaledEntradaX = scaler.transform(X_entrada)
print(rescaledEntradaX)

In [ ]:
# Predição de classes dos dados de entrada

saidas = model.predict(rescaledEntradaX)
print(saidas)

Durante a simulação, foram incluídos novos dados para avaliar o modelo, onde o mesmo trouxe com sucesso o resultado: [1, 0, 1].

Dessa forma, foi possível prever que a segunda franquia não precisaria de abastecimento.

No entanto, existiria a necessidade da primeira e a terceira franquia repor seus estoques.

### **Conclusão**

- Durante os primeiros testes realizados, o atributo *estoque_loja* foi utilizado para avaliação dos modelos, o que consequentemente causou overfitting em 3 dos 5 modelos utilizados: DecisionTree, RandomForest e GradientBoosting, retornando 100% cada um. Já os modelos SVM e ExtraTrees, retornaram cerca de 58% de acurácia.

- Ao utilizar 2 dos 3 atributos obtidos por meio do processo de Feature Selection, foi possível determinar melhores resultados entre os modelos.

- A utilização da estratificação com holdout permitiu que o modelo fosse avaliado de forma justa e imparcial, garantindo a capacidade de generalizar adequadamente para novos dados inseridos posteriormente.

- Apesar de apresentar 2 outliers previstos para cima, o RandomForest se mostrou com melhor resultado durante os testes, apresentando 72,99% de acurácia.